####  This notebook is to test for association between microbiome composition and sleep measurements controlling confounding factors (MiRKAT)

In [48]:
library(MiRKAT)

In [49]:
dat <- read.delim("mapping_sleep_alpha.txt")
head(dat)

X.SampleID,BarcodeSequence,LinkerPrimerSequence,Experiment_Design_Description,Library_Construction_Protocol,Linker,Platform,Center_Name,Center_Project,Instrument_Model,...,M1ADEPR,M1BENZO,AMAMPT_C1,AMFVT_C1,AMPHIT_15SD,Description,alpha_pd,observed_otus,shannon,pielou_e
BI0023,TCTGGTGACATT,GGACTACHVGGGTWTCTAAT,16S stool samples sequenced for MrOS Vitamin D study,16S rRNA v4,GT,Illumina,BI,MrOS,Illumina MiSeq,...,0: No,0: No,Missing:not collected,Missing:not collected,Missing:not collected,Orwoll.BI0023.BI,27.77117,302,5.727116,0.6951729
BI0056,CAAGCATGCCTA,GGACTACHVGGGTWTCTAAT,16S stool samples sequenced for MrOS Vitamin D study,16S rRNA v4,GT,Illumina,BI,MrOS,Illumina MiSeq,...,0: No,0: No,1.0,1.0,0.0,Orwoll.BI0056.BI,17.93266,173,3.888281,0.5229960
BI0131,CTATTTGCGACA,GGACTACHVGGGTWTCTAAT,16S stool samples sequenced for MrOS Vitamin D study,16S rRNA v4,GT,Illumina,BI,MrOS,Illumina MiSeq,...,0: No,0: No,1.0,1.0,0.0,Orwoll.BI0131.BI,21.45366,223,4.050730,0.5192645
BI0153,ATCGGCGTTACA,GGACTACHVGGGTWTCTAAT,16S stool samples sequenced for MrOS Vitamin D study,16S rRNA v4,GT,Illumina,BI,MrOS,Illumina MiSeq,...,0: No,0: No,1.0,1.0,-1.0,Orwoll.BI0153.BI,18.46968,223,4.894253,0.6273959
BI0215,CCTCTCGTGATC,GGACTACHVGGGTWTCTAAT,16S stool samples sequenced for MrOS Vitamin D study,16S rRNA v4,GT,Illumina,BI,MrOS,Illumina MiSeq,...,0: No,0: No,1.0,0.0,-1.0,Orwoll.BI0215.BI,20.04983,222,5.295055,0.6793396
BI0353,TGCCATCTGAAT,GGACTACHVGGGTWTCTAAT,16S stool samples sequenced for MrOS Vitamin D study,16S rRNA v4,GT,Illumina,BI,MrOS,Illumina MiSeq,...,0: No,0: No,1.0,1.0,0.0,Orwoll.BI0353.BI,13.75183,155,4.213016,0.5790192


In [50]:
distance <- read.table(file = 'distance-matrix.tsv', sep = '\t', header = TRUE, row.names = 1)
distance <- distance[ , order(colnames(distance))]
distance <- distance[ order(rownames(distance)), ]
distance[] <- lapply(distance, as.numeric)
head(distance)

,BI0023,BI0056,BI0131,BI0153,BI0215,BI0353,BI0371,BI0372,BI0380,BI0389,...,SD8964,SD8966,SD8972,SD8973,SD8985,SD8996,SD8999,SD9001,SD9003,SD9009
BI0023,0.000000,3.854117,2.2624907,2.4775439,2.5230138,1.7619373,2.1528977,1.9830176,2.6162961,3.2161124,...,2.580809,2.363842,1.8658996,0.6541107,2.0763756,2.4418900,2.1524018,2.4828776,1.4064145,1.9565111
BI0056,3.854117,0.000000,1.9646308,1.5220292,1.3861677,2.2796945,2.1249924,1.9383154,1.3795474,0.7936802,...,2.497952,2.510412,2.1737659,3.3408064,1.8774769,1.6105415,1.7734031,1.6163271,2.6733278,2.0245220
BI0131,2.262491,1.964631,0.0000000,0.4890927,1.1557511,1.4876324,0.2544336,1.0246636,0.7861452,1.2313847,...,2.504723,1.458492,0.6172946,1.6397262,0.4333382,1.4194710,1.1651333,0.5942780,0.8830507,0.4597581
BI0153,2.477544,1.522029,0.4890927,0.0000000,0.7256925,1.3117906,0.6347898,0.8148648,0.3597706,0.7980843,...,2.188765,1.576987,0.7198735,1.9029710,0.4015967,1.0378047,0.8612151,0.4745573,1.1733543,0.5377071
BI0215,2.523014,1.386168,1.1557511,0.7256925,0.0000000,0.9155407,1.2265470,0.5707754,0.5491120,0.8728960,...,1.521053,1.914718,1.0009216,2.0749441,0.8104914,0.3478145,0.3933339,0.9894309,1.4846073,0.9068951
BI0353,1.761937,2.279694,1.4876324,1.3117906,0.9155407,0.0000000,1.4634691,0.5320803,1.2680218,1.7699147,...,1.131693,1.997822,1.0184174,1.4855149,1.0649812,0.7169299,0.5315208,1.4751000,1.2178086,1.0416172


In [51]:
vars_test = c('AMAMPT','AMPHIT','AMFVT','PQPEFFCY', 'PQPSQI','AMAMPT_C1' , 'AMFVT_C1','PQBADSLP')
vars_imp = c('GIERACE', 'SITE','MHDIAB', 'M1ADEPR', 'PASCORE',"BMI")
dat <- dat[c(vars_test,vars_imp)]
head(dat)

AMAMPT,AMPHIT,AMFVT,PQPEFFCY,PQPSQI,AMAMPT_C1,AMFVT_C1,PQBADSLP,GIERACE,SITE,MHDIAB,M1ADEPR,PASCORE,BMI
Missing:not collected,Missing:not collected,Missing:not collected,93.33,4,Missing:not collected,Missing:not collected,0: No,1:WHITE,Birmingham,0: No,0: No,91.00000,28.89012
176.1758554,11.9598842,1049.2338550000002,60.0,9,1.0,1.0,1: Yes,1:WHITE,Birmingham,0: No,0: No,199.17857,28.53980
271.7428785,10.99705997,2069.032006,90.0,2,1.0,1.0,0: No,1:WHITE,Birmingham,0: No,0: No,161.71429,25.01424
190.69266919999998,10.51493608,1498.285048,100.0,3,1.0,1.0,0: No,1:WHITE,Birmingham,0: No,0: No,88.21429,30.87637
240.3133519,10.83951835,702.4915457000001,88.89,1,1.0,0.0,0: No,1:WHITE,Birmingham,0: No,0: No,256.82143,33.58739
345.2370838,12.17318952,1836.677682,80.0,6,1.0,1.0,1: Yes,1:WHITE,Birmingham,0: No,0: No,179.57143,26.41523


In [52]:
dat$AMFVT <- as.numeric(as.character(dat$AMFVT))
dat$PQPEFFCY <- as.numeric(as.character(dat$PQPEFFCY))

ind <- which(is.na(dat$AMFVT))
ind2 <- which(is.na(dat$PQPEFFCY))
dist <- distance[-c(ind,ind2), ]
dist <- dist[, -c(ind,ind2)]
dat <- na.omit(dat)
head(dat)

Warning message in eval(expr, envir, enclos):
"NAs introduced by coercion"Warning message in eval(expr, envir, enclos):
"NAs introduced by coercion"

,AMAMPT,AMPHIT,AMFVT,PQPEFFCY,PQPSQI,AMAMPT_C1,AMFVT_C1,PQBADSLP,GIERACE,SITE,MHDIAB,M1ADEPR,PASCORE,BMI
2,176.1758554,11.9598842,1049.2339,60.00,9,1.0,1.0,1: Yes,1:WHITE,Birmingham,0: No,0: No,199.17857,28.53980
3,271.7428785,10.99705997,2069.0320,90.00,2,1.0,1.0,0: No,1:WHITE,Birmingham,0: No,0: No,161.71429,25.01424
4,190.69266919999998,10.51493608,1498.2850,100.00,3,1.0,1.0,0: No,1:WHITE,Birmingham,0: No,0: No,88.21429,30.87637
5,240.3133519,10.83951835,702.4915,88.89,1,1.0,0.0,0: No,1:WHITE,Birmingham,0: No,0: No,256.82143,33.58739
6,345.2370838,12.17318952,1836.6777,80.00,6,1.0,1.0,1: Yes,1:WHITE,Birmingham,0: No,0: No,179.57143,26.41523
7,153.4988362,15.24806835,514.8809,88.89,5,1.0,0.0,0: No,2:AFRICAN AMERICAN,Birmingham,0: No,0: No,144.57143,28.39116


In [53]:
dim(dat)
head(dist)

[1] 417  14

,BI0056,BI0131,BI0153,BI0215,BI0353,BI0371,BI0372,BI0380,BI0389,BI0395,...,SD8918,SD8934,SD8942,SD8966,SD8972,SD8973,SD8985,SD8996,SD9001,SD9009
BI0056,0.000000,1.9646308,1.5220292,1.3861677,2.2796945,2.1249924,1.9383154,1.3795474,0.7936802,1.7095180,...,0.5956897,3.898778,3.097856,2.510412,2.1737659,3.340806,1.8774769,1.6105415,1.6163271,2.0245220
BI0131,1.964631,0.0000000,0.4890927,1.1557511,1.4876324,0.2544336,1.0246636,0.7861452,1.2313847,1.9934037,...,1.6046783,2.606488,1.890094,1.458492,0.6172946,1.639726,0.4333382,1.4194710,0.5942780,0.4597581
BI0153,1.522029,0.4890927,0.0000000,0.7256925,1.3117906,0.6347898,0.8148648,0.3597706,0.7980843,1.5867218,...,1.1208936,2.726822,1.925176,1.576987,0.7198735,1.902971,0.4015967,1.0378047,0.4745573,0.5377071
BI0215,1.386168,1.1557511,0.7256925,0.0000000,0.9155407,1.2265470,0.5707754,0.5491120,0.8728960,0.8659662,...,0.8151901,2.661172,1.763694,1.914718,1.0009216,2.074944,0.8104914,0.3478145,0.9894309,0.9068951
BI0353,2.279694,1.4876324,1.3117906,0.9155407,0.0000000,1.4634691,0.5320803,1.2680218,1.7699147,1.0309972,...,1.7060332,1.897092,0.926741,1.997822,1.0184174,1.485515,1.0649812,0.7169299,1.4751000,1.0416172
BI0371,2.124992,0.2544336,0.6347898,1.2265470,1.4634691,0.0000000,1.0035298,0.8492519,1.3599767,2.0455402,...,1.7314988,2.586406,1.841603,1.646293,0.4837797,1.542577,0.5014978,1.4526540,0.8340428,0.4355701


In [54]:
test <- dat[c(vars_test)]
head(test)

,AMAMPT,AMPHIT,AMFVT,PQPEFFCY,PQPSQI,AMAMPT_C1,AMFVT_C1,PQBADSLP
2,176.1758554,11.9598842,1049.2339,60.00,9,1.0,1.0,1: Yes
3,271.7428785,10.99705997,2069.0320,90.00,2,1.0,1.0,0: No
4,190.69266919999998,10.51493608,1498.2850,100.00,3,1.0,1.0,0: No
5,240.3133519,10.83951835,702.4915,88.89,1,1.0,0.0,0: No
6,345.2370838,12.17318952,1836.6777,80.00,6,1.0,1.0,1: Yes
7,153.4988362,15.24806835,514.8809,88.89,5,1.0,0.0,0: No


In [55]:
imp <- dat[c(vars_imp)]
head(imp)

,GIERACE,SITE,MHDIAB,M1ADEPR,PASCORE,BMI
2,1:WHITE,Birmingham,0: No,0: No,199.17857,28.53980
3,1:WHITE,Birmingham,0: No,0: No,161.71429,25.01424
4,1:WHITE,Birmingham,0: No,0: No,88.21429,30.87637
5,1:WHITE,Birmingham,0: No,0: No,256.82143,33.58739
6,1:WHITE,Birmingham,0: No,0: No,179.57143,26.41523
7,2:AFRICAN AMERICAN,Birmingham,0: No,0: No,144.57143,28.39116


In [56]:
lapply(test, class)
lapply(imp, class)

$AMAMPT
[1] "factor"

$AMPHIT
[1] "factor"

$AMFVT
[1] "numeric"

$PQPEFFCY
[1] "numeric"

$PQPSQI
[1] "numeric"

$AMAMPT_C1
[1] "factor"

$AMFVT_C1
[1] "factor"

$PQBADSLP
[1] "factor"

$GIERACE
[1] "factor"

$SITE
[1] "factor"

$MHDIAB
[1] "factor"

$M1ADEPR
[1] "factor"

$PASCORE
[1] "numeric"

$BMI
[1] "numeric"

In [57]:
imp[] <- lapply(imp, as.numeric)
test[, 1:5] <- lapply(test[, 1:5], as.character)
test[, 1:5] <- lapply(test[, 1:5], as.numeric)
test[, 6:8] <- lapply(test[, 6:8], as.factor)
test[, 6:8] <- lapply(test[, 6:8], as.numeric)
test[, 6:8] <- sapply(c(6:8), function(i){
    a <- as.numeric(test[, i])
    ifelse(a==2, 1, 0)})
head(test)

,AMAMPT,AMPHIT,AMFVT,PQPEFFCY,PQPSQI,AMAMPT_C1,AMFVT_C1,PQBADSLP
2,176.1759,11.95988,1049.2339,60.00,9,1,1,1
3,271.7429,10.99706,2069.0320,90.00,2,1,1,0
4,190.6927,10.51494,1498.2850,100.00,3,1,1,0
5,240.3134,10.83952,702.4915,88.89,1,1,0,0
6,345.2371,12.17319,1836.6777,80.00,6,1,1,1
7,153.4988,15.24807,514.8809,88.89,5,1,0,0


In [58]:
head(dist)

,BI0056,BI0131,BI0153,BI0215,BI0353,BI0371,BI0372,BI0380,BI0389,BI0395,...,SD8918,SD8934,SD8942,SD8966,SD8972,SD8973,SD8985,SD8996,SD9001,SD9009
BI0056,0.000000,1.9646308,1.5220292,1.3861677,2.2796945,2.1249924,1.9383154,1.3795474,0.7936802,1.7095180,...,0.5956897,3.898778,3.097856,2.510412,2.1737659,3.340806,1.8774769,1.6105415,1.6163271,2.0245220
BI0131,1.964631,0.0000000,0.4890927,1.1557511,1.4876324,0.2544336,1.0246636,0.7861452,1.2313847,1.9934037,...,1.6046783,2.606488,1.890094,1.458492,0.6172946,1.639726,0.4333382,1.4194710,0.5942780,0.4597581
BI0153,1.522029,0.4890927,0.0000000,0.7256925,1.3117906,0.6347898,0.8148648,0.3597706,0.7980843,1.5867218,...,1.1208936,2.726822,1.925176,1.576987,0.7198735,1.902971,0.4015967,1.0378047,0.4745573,0.5377071
BI0215,1.386168,1.1557511,0.7256925,0.0000000,0.9155407,1.2265470,0.5707754,0.5491120,0.8728960,0.8659662,...,0.8151901,2.661172,1.763694,1.914718,1.0009216,2.074944,0.8104914,0.3478145,0.9894309,0.9068951
BI0353,2.279694,1.4876324,1.3117906,0.9155407,0.0000000,1.4634691,0.5320803,1.2680218,1.7699147,1.0309972,...,1.7060332,1.897092,0.926741,1.997822,1.0184174,1.485515,1.0649812,0.7169299,1.4751000,1.0416172
BI0371,2.124992,0.2544336,0.6347898,1.2265470,1.4634691,0.0000000,1.0035298,0.8492519,1.3599767,2.0455402,...,1.7314988,2.586406,1.841603,1.646293,0.4837797,1.542577,0.5014978,1.4526540,0.8340428,0.4355701


In [59]:
ker <- D2K(as.matrix(dist))

In [60]:
set.seed(10)
MiRKAT(test$AMFVT, imp, ker, out_type = "C", method = "permutation")
MiRKAT(test$AMFVT_C1, imp, ker, out_type = "D", method = "permutation")

[1] 0.026

[1] 0.159

In [61]:
set.seed(10)
MiRKAT(test$AMAMPT, imp, ker, out_type = "C", method = "permutation")
MiRKAT(test$AMAMPT_C1, imp, ker, out_type = "D", method = "permutation")

[1] 0.886

[1] 0.86

In [62]:
set.seed(10)
MiRKAT(test$AMPHIT, imp, ker, out_type = "C", method = "permutation")

[1] 0.71

In [63]:
set.seed(10)
MiRKAT(test$PQPEFFCY, imp, ker, out_type = "C", method = "permutation")
MiRKAT(test$PQBADSLP, imp, ker, out_type = "D", method = "permutation")
MiRKAT(test$PQPSQI, imp, ker, out_type = "C", method = "permutation")

[1] 0.053

[1] 0.169

[1] 0.305